In [ ]:
import gzip
import os

import cudf
from cuml import TruncatedSVD as cumlTSVD
import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD as skTSVD
from sklearn.metrics import mean_squared_error

# Helper Functions

In [ ]:
def load_data(nrows, ncols, cached="data/mortgage.npy.gz"):
    """Load mortgage dataset if it is available, otherwise use random data."""
    if os.path.exists(cached):
        print("Using mortgage data.")
        with gzip.open(cached) as f:
            X = np.load(f)
        X = X[np.random.randint(0, X.shape[0] - 1, nrows), :ncols]
    else:
        print("Using random data")
        X = np.random.rand(nrows, ncols)
    df = pd.DataFrame({"fea%d" % i: X[:, i] for i in range(X.shape[1])})
    return df

In [ ]:
def array_equal(a, b, threshold=5e-3, with_sign=True):
    """Check the equality of two arrays with a given threshold."""
    a = to_nparray(a)
    b = to_nparray(b)
    if not with_sign:
        a, b = np.abs(a), np.abs(b)
    error = mean_squared_error(a, b)
    res = error < threshold
    return res

def to_nparray(x):
    """Convert a given array into a numpy array."""
    if isinstance(x, np.ndarray) or isinstance(x, pd.DataFrame):
        return np.array(x)
    elif isinstance(x, np.float64):
        return np.array([x])
    elif isinstance(x, cudf.DataFrame) or isinstance(x, cudf.Series):
        return x.to_pandas().values
    return x

# Run tests

In [ ]:
%%time
nrows = 2**22
ncols = 40

X = load_data(nrows, ncols)
print("Data shape:", X.shape)

In [ ]:
n_components = 10
random_state = 42

In [ ]:
%%time
algorithm = "arpack"
tsvd_sk = skTSVD(n_components=n_components, algorithm=algorithm, 
                 random_state=random_state)
result_sk = tsvd_sk.fit_transform(X)

In [ ]:
%%time
X = cudf.DataFrame.from_pandas(X)

In [ ]:
%%time
algorithm = "full"
tsvd_cuml = cumlTSVD(n_components=n_components,algorithm=algorithm, 
                     random_state=random_state)
result_cuml = tsvd_cuml.fit_transform(X)

In [ ]:
for attr in ["singular_values_", "components_"]:
    passed = array_equal(getattr(tsvd_sk, attr), getattr(tsvd_cuml, attr), threshold=0.1)
    # Larger error margin due to different algorithms: arpack vs full.
    message = "compare tsvd: cuml vs sklearn {} {}".format(
        attr,
        "equal" if passed else "NOT equal"
    )
    print(message)

In [ ]:
passed = array_equal(result_sk, result_cuml, threshold=0.1)
# Larger error margin due to different algorithms: arpack vs full.
message = "compare tsvd: cuml vs sklearn transformed results %s" % \
    ("equal" if passed else "NOT equal")
print(message)